In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pylab as plt
import tensorflow_probability as tfp
import scipy

In [ ]:
#Plot data and true function
x_all = np.linspace(0,1,100)
y_true = x_all**2+.5*x_all-.5

x_data = np.reshape((np.array([0.25,.75])[None]*np.ones((5,))[:,None]),(-1,))
# x_data = np.random.uniform(.1,.5,50)
y_data = (x_data**2+.5*x_data-.5) + np.random.normal(0,.05,len(x_data))

def plot_data():
    plt.plot(x_all,y_true,label='True function')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.plot(x_data,y_data,'ro',label='Data points')
    
plot_data()
plt.legend()

In [ ]:
#Design matrix for data and test points
def makeX(x):
    return np.stack([np.ones(len(x)),x,x**2],axis=-1)
X = makeX(x_data)
print(X)
X_all = makeX(x_all)

In [ ]:
#We can't invert the normal matrix
a = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y_data))

In [ ]:
#One option is to use the pseudoinverse
a_pinv = np.linalg.pinv(X.T.dot(X)).dot(X.T.dot(y_data))
y_pinv = X_all.dot(a_pinv)

plot_data()
plt.plot(x_all,y_pinv,label='Pseudoinverse')
plt.legend()


In [ ]:
#Or we can regularize
a_reg1 = np.linalg.inv(X.T.dot(X)+1e-1*np.eye(3)).dot(X.T.dot(y_data))
a_reg2 = np.linalg.inv(X.T.dot(X)+1e-4*np.eye(3)).dot(X.T.dot(y_data))

y_reg1 = X_all.dot(a_reg1)
y_reg2 = X_all.dot(a_reg2)

plot_data()
plt.plot(x_all,y_reg1,label=r'Regularize: $\alpha = 10^{-1}$')
plt.plot(x_all,y_reg2,label=r'Regularize: $\alpha = 10^{-4}$')
plt.legend()

In [ ]:
#Bayesian inference
sig_l = 1e0
sig_p = 1e2
mu = np.linalg.solve(X.T.dot(X) + sig_l**2/sig_p**2 * np.eye(3),X.T.dot(y_data))
Omega = np.linalg.inv(1./sig_l**2*X.T.dot(X) + 1./sig_p**2 * np.eye(3))

In [ ]:
#Prior predictive plot
nu = X_all.dot(np.zeros_like(mu))
sig_y = X_all.dot(np.eye(3)*sig_p**2).dot(X_all.T)
sig_y_diag = np.sqrt(np.diag(sig_y))
plot_data()
plt.plot(x_all,nu,label = 'Prior predictive mean')
plt.fill_between(x_all,nu-sig_y_diag,nu+sig_y_diag,color='C4',alpha=.3,label=r'95% of confidence')
plt.legend()

In [ ]:
#Posterior predictive plot
nu = X_all.dot(mu)
sig_y = X_all.dot(Omega).dot(X_all.T)
sig_y_diag = np.sqrt(np.diag(sig_y))
plot_data()
plt.plot(x_all,nu,label = 'Posterior predictive mean')
plt.fill_between(x_all,nu-sig_y_diag,nu+sig_y_diag,color='C4',alpha=.3,label=r'95% of confidence')
plt.legend()

In [ ]:
#We can observe two types of uncertainty, aleatoric and epistemic if we add noise to the data
#Aleatoric: inherent stochasticity in the data
#Epistemic: uncertainty due to lack of data